# 3주차, 5일차 : 코드 암기하기
- ### Contents 
    1. Better performance with the tf.data API: https://www.tensorflow.org/guide/data_performance
    2. Time Series Forecasting: https://www.tensorflow.org/tutorials/structured_data/time_series
    3. Text Classification with an RNN: https://www.tensorflow.org/tutorials/text/text_classification_rnn
    4. Distributed training with Keras: https://www.tensorflow.org/tutorials/distribute/keras

## 1. Better performance with the tf.data API (1/3)
### 개요 
GPU와 TPU는 하나의 학습 단계를 실행하는데 필요한 시간을 급격하게 줄일 수 있습니다. <br>
최대 성능을 위해서는 현재 단계가 종료되기 전에 다음 단계의 데이터를 운반하는 효율적인 입력 파이프라인이 필요합니다.<br>
`tf.data` API는 유연하고 효율적인 입력 파이프라인을 만드는데 도움이 됩니다. 

- 텐서플로 입력 파이프라인이 기본적으로 ETL 프로세스라는 것을 설명합니다.
- 고성능 텐서플로 입력 파이프라인을 설계하기 위해 권장하는 방법을 설명합니다. 
- 변환 순서에 따른 성능 영향에 대해 설명합니다.

### 입력 파이프라인 구조
- 전형적인 텐서플로 훈련 입력 파이프라인은 ETL 프로세스로 구성될 수 있습니다.
    1. **추출**: 메모리(Numpy)나 로컬(HDD 또는 SSD)이나 원격(이를테면 GCS or HDFS) 영구 스토리지로부터 데이터를 읽어들입니다.
    2. **변환**: 분석하기 위해 CPU를 사용하여 셔플링, 배칭 그리고 압축 해제와 Augmentation, 텍스트 벡터화 또는 비디오 시간 샘플링과 같은 데이터 전처리를 수행합니다.
    3. **적재**: 변환된 데이터를 가속장치(들)에 적재합니다.

- 이 패턴은 CPU를 효과적으로 사용하면서 모델 훈련을 많이 수행하도록 가속기를 예비합니다. 게다가 입력 파이프라인을 ETL 프로세스로 보는 것은 성능 최적화를 쉽게 적용할 수 있는 프레임워크를 제공합니다.
- 아래 예제는 레이블된 이미지가 포함된 TFRecord 파일들을 읽고 이를 학습에 적합한 이미지-레이블 쌍의 배치(batch)로 변환하는 입력 파이프라인의 간단한 구현 입니다. 
- 입력 파이프라인은 `tf.data.Dataset`로 표현되고 `tf.keras`와 같은 고수준의 텐서플로 API에 전달될 수 있습니다.

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import time

### The Dataset

In [10]:
'''
이번 가이드에서는 데이터셋과 성능 측정을 반복해서 번갈아 살펴볼텐데, 재현 가능한 성능 벤치마크는 다음과 같은 이유로 만들기 어려울겁니다.
1. 현재 CPU 부하
2. 네트워크 트레픽
3. 캐시 같은 복잡한 메커니즘에 의해서
그래서 재현 가능한 벤치마크를 위해 인공적인 데이터셋을 사용하겠습니다.
tf.data.Dataset을 상속받아 AritificialDataset이라는 클래스를 정의하겠습니다. 
이 데이터셋은 
- 'num_sample' 만큼 샘플을 생성합니다.
- 파일에서 첫번째 아이템이 시뮬레이션 하기 전까지 sleep 합니다.
- 각 아이템이 생성되어 파일로부터 데이터를 읽는 걸 시뮬레이션 하기 전까지 sleep 합니다.
'''

class ArtificialDataset(tf.data.Dataset):
    def _generator(num_samples):
        #Opening the file
        time.sleep(0.03)
        
        for sample_idx in range(num_samples):
            time.sleep(0.015)
            
            yield (sample_idx,)
    
    def __new__(cls, num_samples=3):
        return tf.data.Dataset.from_generator(
            cls._generator,
            output_types=tf.dtypes.int64,
            output_shapes=(1,),
            args=(num_samples,)
        )

### The training loop

In [11]:
'''
데이터셋 전체를 반복하기까지 얼마나 오래 걸리는지 측정하기 위한 더미 학습 루프를 작성합니다.
'''
def benchmark(dataset, num_epochs=2):
    start_time = time.perf_counter()
    for epoch_num in range(num_epochs):
        for sample in dataset:
            # Performong a training step
            time.sleep(0.01)
    tf.print('Execution time:', time.perf_counter() - start_time)

### Optimize performance
- 어떻게 성능이 최적화 될 수 있는지 보여주기 위해, `ArtificialDatast`의 성능을 향상시켜보겠습니다.

#### The naive approach
- 트릭 없이 나이브한 파이프라인으로 시작해봅니다. as-is 데이터셋을 반복시켜봅니다.

In [12]:
benchmark(ArtificialDataset())

Execution time: 0.23924585599979764


- 실행 시간이 이런식으로 수행되었었다.

<img src= https://www.tensorflow.org/guide/images/data_performance/naive.svg/ >

- Training step에 포함 되어 있는 것들
    1. 아직 열리지 않은 파일을 열기
    2. 파일로부터 데이터 엔트리 가져오기
    3. 학습에 데이터 사용하기
    
- 하지만 이러한 동기화된 구현은, 파이프라인이 데이터를 불러오는 동안, 모델은 유휴 상태에 있게됩니다. 반대로 모델이 학습하는 동안에는 입력 파이프라인이 유휴상태에 있게 됩니다. 학습 과정 시간은 이러한 것들의 합입니다. 파일 열기, 읽기, 학습 시간
- 다음에는 입력 파이프라인에는 최적으로 디자인된 텐서플로우 입력 파이프라인 수행자를 사용해보겠습니다. 

### Prefetching
- Prefetching은 전처리와 모델 학습 수행을 겹치도록 하는 것입니다. 모델이 학습을 수행하는 동안 입력 파이프라인은 데이터를 읽어옵니다. 이렇게하면, 학습하는 시간과 추출하는 시간을 최대한 줄일 수 있습니다. 

In [13]:
benchmark(
    ArtificialDataset()
    .prefetch(tf.data.experimental.AUTOTUNE)
)

Execution time: 0.18514143799984595


<img src=https://www.tensorflow.org/guide/images/data_performance/prefetched.svg />

이제 샘플 0에 대해 학습하고 있을 때, 입력 파이프라인은 샘플 1 데이터를 읽고 있습니다. 

### Parallelizing data extraction 
- 실제 세계에서는, 입력 데이터는 원격에 저장되어 있습니다. 데이터셋 파이프라인은 로컬에서 데이터를 읽을 때 좋습니다. 만약에 데이터가 원격 저장소에 있다면 I/O 병목 현상이 일어나게 됩니다. 이러한 현상은 로컬 원격 저장소의 차이점에 의해 생깁니다.
    - Time-to-first-byte: 파일의 첫번째 바이트 부터 읽는 것은 원격 저장소가 로컬 저장소보다 오래 걸립니다. 
    - Read throughput: 원격 저장소가 일반적으로 넓은 대역폭을 제공하더라도, 단일 파일을 읽는 것은 대역폭의 작은 부분만 활용합니다.
    - 여러가지 데이터 추출 오버헤드를 완화 하기 위해, `tf.data.Dataset.interleave` 전송을 병렬적으로 사용할 수 있습니다. 

In [14]:
benchmark(tf.data.Dataset.range(2)
         .interleave(ArtificialDataset))

Execution time: 0.1899094119999063


- 다음 그림은 `interleave` 전송을 보여줍니다. 하지만 성능 향상이 포함되지는 않았습니다.
<img src=https://www.tensorflow.org/guide/images/data_performance/sequential_interleave.svg />

#### 병렬 인터리브
- 이제 `num_parallel_calls` 파라미터를 사용하겠습니다. 다중 데이터셋을 병렬적으로 불러옵니다,

In [15]:
benchmark(
    tf.data.Dataset.range(2)
    .interleave(
        ArtificialDataset,
        num_parallel_calls=tf.data.experimental.AUTOTUNE
    )
)

Execution time: 0.12735241500013217


<img src=https://www.tensorflow.org/guide/images/data_performance/parallel_interleave.svg />
- 이제야 두개의 데이터셋을 병렬적으로 읽어올 수 있었습니다. 전체 데이터 처리 시간을 줄였습니다.

### 데이터 변환 병렬화하기
- 데이터를 준비할 때, 입력 성분은 전처리가 필요할 수 있습니다. `tf.data`API는 `tf.data.Dataset.map` 변환을 제공하며, 유저-정의 함수를 각 원소에 적용할 수 있습니다. 
- 각 데이터셋 원소는 서로 독립적이기 때문에 이러한 전처리는 다중 GPU를 통해 병렬적으로 처리할 수 있습니다. 
- 이는 `prefetch`와 `interleave` 변환과 비슷하게 가능하도록 만들 수 있습니다.
- `map` 변환은 병렬 수준을 지정할 수 있는 `num_parallel_call` 인수를 제공합니다.

In [16]:
def mapped_function(s):
    # 하드 전처리를 진행합니다.
    tf.py_function(lambda: time.sleep(0.04), [], ())
    return s

#### Sequential Mapping
- 병렬화 없이 `map` 변환을 사용해보겠습니다.

In [17]:
benchmark(ArtificialDataset()
         .map(mapped_function))

Execution time: 0.47504732699962915


<img src= https://www.tensorflow.org/guide/images/data_performance/sequential_map.svg />
- 나이브한 접근으로 단일 이터레이션에 대한 열기, 읽기, 전처리 그리고 학습에 대한 전체 수행 시간이 나와있습니다.

#### Parallel mapping
- 그럼 이제 다중 샘플에 같은 전처리 함수를 병렬적으로 적용하겠습니다.

In [18]:
benchmark(
    ArtificialDataset()
    .map(
        mapped_function,
        num_parallel_calls=tf.data.experimental.AUTOTUNE
    )
)

Execution time: 0.26811543900021206


<img src=https://www.tensorflow.org/guide/images/data_performance/parallel_map.svg />
- 이제 전처리 단계가 겹쳐져 단일 이터레이션의 전체 시간이 줄어든 것을 확인할 수 있습니다.

#### Caching
